# Now Let's put all the pieces together


In [9]:
!pip install -U llama-stack-client==0.2.5 dotenv > /dev/null 2>&1 && echo "pip Python Prerequisites installed succesfuly"

import os
# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

# for communication with Llama Stack
from llama_stack_client import LlamaStackClient

# These libraries are just here to print the results from the agent in a more human-readable way 
from src.utils import step_printer
#from src.client_tools import get_location
from termcolor import cprint
import uuid
from llama_stack_client.lib.agents.event_logger import EventLogger
stream=False ## Defaulting to False, you can change this throughout the section to "True" if you wanted to see the output in another format (Using EventLogger)

# for our lab, we will just define our variables manualy here, in a regular application, this would be ready directly from the local .env file and we would comment these lines out
os.environ['LLAMA_STACK_SERVER'] = 'http://localhost:8321'
from llama_stack_client import LlamaStackClient
LLAMA_STACK_SERVER=os.getenv("LLAMA_STACK_SERVER")

client = LlamaStackClient(
    base_url=LLAMA_STACK_SERVER,
    provider_data=provider_data
)

# List available models and select from allowed models list
allowed_models_list=["granite3.2:8b"]
selected_model = None

models = client.models.list()
print("--- Available models: ---")
for m in models:
    print(f"{m.identifier} - {m.provider_id} - {m.provider_resource_id}")
    # Check if the model identifier contains any of the allowed substrings
    if any(substring in m.identifier for substring in allowed_models_list):
        # Only set selected_model if it hasn't been set yet
        if selected_model is None:
            selected_model = m.identifier
           
# If no allowed model was found, you might want to handle that case
if selected_model is None:
    print("No allowed model found in the list.")
print(f"Selected model (from allowed list): {selected_model}")
model = selected_model

vector_db_id = "Our_Parks_DB"
query_config = {
    "query_generator_config": {
        "type": "default",
        "separator": " "
    },
    "max_tokens_in_context": 300,
    "max_chunks": 2
}


pip Python Prerequisites installed succesfuly
--- Available models: ---
all-MiniLM-L6-v2 - ollama - all-minilm:latest
granite3.2:8b - ollama - granite3.2:8b
meta-llama/Llama-3.2-3B-Instruct - ollama - llama3.2:3b-instruct-fp16
Selected model (from allowed list): granite3.2:8b


ok, now we are going to do some strange things, just for the sake of learning, We are going to create our own prompt, and then, populate it with the list of tools and their description, to teach our ReAct agent what they can do and how we want them to behave. 

If you are curious to see the original, default prompt, you can find it here: https://github.com/meta-llama/llama-stack-client-python/blob/main/src/llama_stack_client/lib/agents/react/prompts.py

In [17]:
custom_react_prompt = """

You are an expert assistant who can solve any task using tool calls. You will be given a task to solve as best you can.

To do so, you have been given access to the following tools: <<tool_names>>

🚨 TOOL USAGE RULES — FOLLOW STRICTLY 🚨

1. ❌ Do NOT guess or invent tool names.
2. ✅ You may only call tools that are explicitly listed in <<tool_names>>.
3. 🛑 Never use tools like `google_maps`, `search`, `maps_geocode`, or `knowledge_search` unless they are explicitly in <<tool_names>>.
4. If you cannot solve a task using the tools available, explain that limitation in your final answer instead of calling an invalid tool.
5. 🔎 ALWAYS ALWAYS ALWAYS USE `get_park_location` before `get_alerts`
6. ❓ If a tool needs location data (like `get_alerts`, `maps_search_places`), you must first call `get_park_location` and extract the relevant state, city, or coordinates from its result before proceeding. You must not hardcode, assume, or guess this data.
7. 🧠 Internally keep track of whether you have already retrieved park location. Do not call `get_alerts` until this location has been confirmed via `get_park_location`.

🛦 TOOL PARAMETER FORMAT (MANDATORY):
Each tool call must use the following format for `tool_params`:
```json
"tool_params": [
  {"name": "parameter_name", "value": "actual value"}
]
```

---

🧐 RESPONSE FORMAT (ALWAYS JSON):

{
    "thought": $THOUGHT_PROCESS,
    "action": {
        "tool_name": $TOOL_NAME,
        "tool_params": $TOOL_PARAMS
    },
    "answer": $ANSWER
}

Only one tool may be called at a time. Use multiple steps when needed.

Use `"action": null` and set `answer` when you’re ready to respond to the user.

---

🗓️ EXAMPLE:

Task: “Are there any supermarkets near Crimson Basin?”

Step 1:
{
    "thought": "I need to get the location of Crimson Basin park using get_park_location.",
    "action": {
        "tool_name": "get_park_location",
        "tool_params": [
            {"name": "park_name", "value": "Crimson Basin"}
        ]
    },
    "answer": null
}

Observation: {"result": "Crimson Basin is located in Nevada, USA."}

Step 2:
{
    "thought": "Now I will search for supermarkets near that location using maps_search_places.",
    "action": {
        "tool_name": "maps_search_places",
        "tool_params": [
            {"name": "query", "value": "supermarkets near Nevada, USA"}
        ]
    },
    "answer": null
}

Observation: { ... list of stores ... }

Final step:
{
    "thought": "I now have the info the user asked for.",
    "action": null,
    "answer": "There are several supermarkets near Crimson Basin, including WinCo Foods, Albertsons, and Smith’s."
}

---

❗ IF A TOOL IS MISSING:

If no tool is available for the task:
{
    "thought": "I need to get nearby supermarkets, but I do not have access to a maps tool.",
    "action": null,
    "answer": "I’m unable to find supermarkets near Crimson Basin because I don’t have access to a maps tool."
}

---

You only have access to the following tools: <<tool_descriptions>>

SUMMARY:
- Do not guess tools. Use only the ones listed.
- Follow the exact `tool_params` format.
- Use get_park_location to resolve any location before calling tools that require geographic input.
- Do not hardcode, guess, or assume state names or coordinates.
- Always use get_park_location before get_alerts.
- Keep internal memory of what locations have already been retrieved.
- Return final answers with `"action": null`.

Now Begin! If you solve the task correctly, you will receive a reward of $1,000,000.

"""


### Define which tools we want to inject into the prompt
Here, we extract the tools information from llama stack registed tools (MCPs in our case), notice, that once the MCP servers were registered with Llama Stack Server, it interacts with them and extracts all the available tools, their descriptions and how to use them.

In [26]:
registered_tools = client.tools.list()


allowed_toolgroups = [
    'mcp::mcp-weather',
    'mcp::mcp-googlemaps',
    'mcp::mcp-parks-info',
]

# Initialize an empty list to store the dictionaries for each tool
allowed_tools_array = []

for tool in registered_tools:
    if tool.toolgroup_id in allowed_toolgroups:
        #print(tool)
        allowed_tools_array.append(tool)

# If you want to see what this looks like, uncomment these lines
#print("List of allowed tools (each as a dictionary):")
#print(allowed_tools_array)


### Inject the tools information into the prompt
This usually happens behind the scenes, but in our case, we want to show how we can insert the tools in any way we like.

In [32]:
import re

def insert_tools_to_prompt(my_instructions: str, allowed_tools_array: list) -> str:
    """
    Formats the source template string by inserting tool names and descriptions
    from a list of Tool objects.

    Args:
        my_instructions: A multi-string variable containing the source template.
                         Expected to have <<tool_names>> and <<tool_descriptions>> placeholders.
        allowed_tools_array: A list of Tool objects obtained from the client library.

    Returns:
        A multi-string variable with placeholders replaced by formatted tool information.
    """
    tool_names = []
    tool_descriptions = []

    for tool in allowed_tools_array:
        tool_names.append(tool.identifier)
        formatted_parameters = []
        for param in tool.parameters:
            # Escape single quotes in the parameter description for the string literal
            param_description_escaped = param.description.replace("'", "\\'")
            formatted_parameters.append(
                f"Parameter(description='{param_description_escaped}', name='{param.name}', parameter_type='{param.parameter_type}', required={param.required}, default={param.default})"
            )
        cleaned_description = tool.description.replace('\\n', '\n').replace("    ", "").replace("'", "\\'")

        tool_descriptions.append(
            f"- {tool.identifier}: {{'name': '{tool.identifier}', 'description': '{cleaned_description}', 'parameters': [{', '.join(formatted_parameters)}]}}"
        )

    tool_names_string = ", ".join(tool_names)

    tool_descriptions_string = "\n".join(tool_descriptions)

    output_template = my_instructions.replace("<<tool_names>>", tool_names_string).replace("<<tool_descriptions>>", tool_descriptions_string)

    return output_template


In [33]:
custom_react_prompt_with_tools=insert_tools_to_prompt(custom_react_prompt,allowed_tools_array)
print(custom_react_prompt_with_tools)



You are an expert assistant who can solve any task using tool calls. You will be given a task to solve as best you can.

To do so, you have been given access to the following tools: get_alerts, get_forecast, maps_geocode, maps_reverse_geocode, maps_search_places, maps_place_details, maps_distance_matrix, maps_elevation, maps_directions, get_park_location, get_park_cost, get_park_description, get_park_camping_sites, get_park_seasonal_operations, get_park_seasonal_attractions, get_park_other_information

🚨 TOOL USAGE RULES — FOLLOW STRICTLY 🚨

1. ❌ Do NOT guess or invent tool names.
2. ✅ You may only call tools that are explicitly listed in get_alerts, get_forecast, maps_geocode, maps_reverse_geocode, maps_search_places, maps_place_details, maps_distance_matrix, maps_elevation, maps_directions, get_park_location, get_park_cost, get_park_description, get_park_camping_sites, get_park_seasonal_operations, get_park_seasonal_attractions, get_park_other_information.
3. 🛑 Never use tools like

# Final React Agent
Now, let's explore how our agent ReActs (<-- see what we did there?) 

Important Note: It's important to set your expecations, with different questions, models and prompts and general LLM settings like temperature, our agent will behave differently, this exercise is meant to show you the art of the possible, getting to a "perfect chat client" requires more work and fine tuning. 

In [35]:
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput

stream=False


agent = ReActAgent(
            client=client,
            model=model,
            instructions=tool_populated_instructions,
            tools=["mcp::mcp-parks-info","mcp::mcp-weather","mcp::mcp-googlemaps"],
            response_format={
                "type": "json_schema",
                "json_schema": ReActOutput.model_json_schema(),
            },
            #sampling_params=sampling_params,
        )
user_prompts = [
    #"are there any hotels on the drive from Las Vegas, Nevada to Crimson Basin park",
     "Are there any supermarkets near Crimson Basin park?", # Works
    # "Should I be worried about severe weather conditions around Azure Mongrove Wilderness park today?",
    # "Can you find a pharmacy close to Prismatic Painted Prairie park?",
    # "what are the coordinates of Prismatic Painted Prairie park?",
]


for prompt in user_prompts:
    new_uuid = uuid.uuid4()
    session_id = agent.create_session(f"React-session1-{new_uuid}")
    print("\n"+"="*50)
    print(f"Processing user query: {prompt}", "blue")
    print("="*50)
    print(f"DEBUG: Value of 'stream' variable: {stream}")

    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 



Processing user query: Are there any supermarkets near Crimson Basin park? blue
DEBUG: Value of 'stream' variable: False

---------- 📍 Step 1: InferenceStep ----------
🤖 Model Response:
{
    "thought": "I need to get the location of Crimson Basin park using get_park_location.",
    "action": {
        "tool_name": "get_park_location",
        "tool_params": [
            {"name": "park_name", "value": "Crimson Basin"}
        ]
    },
    "answer": null
}


---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'type': 'text',
│   'text': '{\n  "result": "Crimson Basin is located in Nevada, USA, with coordinates [39.49422066231864, -117.07613301021429]."\n}',
│   'annotations': None
}


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
{
    "thought": "Now I will search for supermarkets near that location using maps_search_places.",
    "action": {
        "tool_name": "maps_search_places",
        "tool_params": [
            {"name": "query", "value": "supermarkets near Nevada, USA"}
        ]
    },
    "answer": null
}


---------- 📍 Step 4: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'type': 'text',
│   'text': '{\n  "places": [\n    {\n      "name": "Cardenas Markets",\n      "formatted_address": "4700 Meadows Ln, Las Vegas, NV 89107, United States",\n      "location": {\n        "lat": 36.1714641,\n        "lng": -115.204666\n      },\n      "place_id": "ChIJbUm9OXHByIARpWnTiD2MpUg",\n      "rating": 4.1,\n      "types": [\n        "supermarket",\n        "grocery_or_supermarket",\n        "food",\n        "point_of_interest",\n        "store",\n        "establishment"\n      ]\n    },\n    {\n      "name": "WinCo Foods",\n      "formatted_address": "80 N Stephanie St, Henderson, NV 89074, United States",\n      "location": {\n        "lat": 36.0337498,\n        "lng": -115.0442349\n      },\n      "place_id": "ChIJW6oSV6fRyIARuvv8kXMi3wc",\n      "rating": 4.6,\n      "types": [\n        "supermarket",\n        "bakery",\n        "grocery_or_supermarket",\n        "food",\n        "point_of_interest",\n        "store",\n        "establishment"\n      ]\n    },\n    {\n      "name": "WinCo Foods",\n      "formatted_address": "9750 S Virginia St, Reno, NV 89511, United States",\n      "location": {\n        "lat": 39.441787,\n        "lng": -119.768291\n      },\n      "place_id": "ChIJTezu-D4VmYARr4h9i2poxiY",\n      "rating": 4.5,\n      "types": [\n        "supermarket",\n        "bakery",\n        "grocery_or_supermarket",\n        "food",\n        "point_of_interest",\n        "store",\n        "establishment"\n      ]\n    },\n    {\n      "name": "WinCo Foods",\n      "formatted_address": "6101 N Decatur Blvd, Las Vegas, NV 89130, United States",\n      "location": {\n        "lat": 36.269792,\n        "lng": -115.20816\n      },\n      "place_id": "ChIJqTkQjcvryIARHCSTjZoqmoA",\n      "rating": 4.6,\n      "types": [\n        "supermarket",\n        "bakery",\n        "grocery_or_supermarket",\n        "food",\n        "point_of_interest",\n        "store",\n        "establishment"\n      ]\n    },\n    {\n      "name": "International Marketplace",\n      "formatted_address": "5000 S Decatur Blvd, Las Vegas, NV 89118, United States",\n      "location": {\n        "lat": 36.0995361,\n        "lng": -115.2068639\n      },\n      "place_id": "ChIJkwS3_IvGyIARJNljqKfLXsE",\n      "rating": 4.3,\n      "types": [\n        "grocery_or_supermarket",\n        "supermarket",\n        "food",\n        "point_of_interest",\n        "store",\n        "establishment"\n      ]\n    },\n    {\n      "name": "WinCo Foods",\n      "formatted_address": "2855 Northtowne Ln, Reno, NV 89512, United States",\n      "location": {\n        "lat": 39.553496,\n        "lng": -119.784073\n      },\n      "place_id": "ChIJXbg6ppg4mYARj0X7oPqSoGc",\n      "rating": 4.2,\n      "types": [\n        "supermarket",\n        "bakery",\n        "grocery_or_supermarket",\n        "food",\n        "point_of_interest",\n        "store",\n        "establishment"\n      ]\n    },\n    {\n      "name": "Albertsons",\n      "formatted_address": "1760 E Charleston Blvd, Las Vegas, NV 89104, United States",\n      "location": {\n        "lat": 36.156893,\n        "lng": -115.1256005\n      },\n      "place_id": "ChIJFSfruGPDyIAR7NCs3wbVRdM",\n      "rating": 4.1,\n      "types": [\n        "grocery_or_supermarket",\n        "florist",\n        "supermarket",\n        "bakery",\n        "food",\n        "point_of_interest",\n        "store",\n        "establishment"\n      ]\n    },\n    {\n      "name": "Albertsons",\n      "formatted_address": "1300 E Flamingo Rd, Las Vegas, NV 89119, United States",\n      "location": {\n        "lat": 36.1158666,\n        "lng": -115.1360834\n      },\n      "place_id": "ChIJK3uGrv7EyIARQrW8rhED9DY",\n      "rating": 4,\n      "types": [\n        "grocery_or_supermarket",\n        "florist",\n        "supermarket",\n        "bakery",\n        "food",\n        "point_of_interest",\n        "store",\n        "establishment"\n      ]\n    },\n    {\n      "name": "Vons",\n      "formatted_address": "1


---------- 📍 Step 5: InferenceStep ----------
🤖 Model Response:
{
  "thought": "The user requested a list of supermarkets in Las Vegas and Reno, Nevada. I have provided a JSON response containing the names, addresses, and ratings of various supermarkets in these cities.",
  "action": null

,
  "answer": "Here is a list of supermarkets in Las Vegas and Reno, Nevada: \n\n1. Cardenas Markets (4 locations) \n2. WinCo Foods (5 locations) \n3. Vons (4 locations) \n4. Mariana's SuperMarkets \n5. El Super \n6. Great Basin Community Food Co-op \n7. Smith's (multiple locations) \n8. La Bonita Supermarkets \n9. Trader Joe's (1 location in Reno)\n\nEach supermarket has multiple addresses within their respective cities, offering a variety of grocery options for residents and visitors."
}

========== Query processing completed ========== 



Summary